In [4]:
import pandas as pd
import numpy as np

In [5]:
df_ex = pd.read_csv("./resources/data/exercise.csv", header=0)
df_workout = pd.read_csv("./resources/data/workout.csv", header=0)
df_user = pd.read_csv("./resources/data/user.csv", header=0)
df_derived_ex = pd.read_csv("./resources/data/exercise_derived.csv", header=0)

In [84]:
# print(df_ex.head(1))
# print(df_ex.dtypes)
# print(df_workout.head(1))
# print(df_workout.dtypes)
# print(df_user.head(1))
# print(df_user.dtypes)
# print(df_derived_ex.head(1))
# print(df_derived_ex.dtypes)

In [85]:
the_columns_exd = df_derived_ex.columns
the_columns_ex = df_ex.columns

In [98]:
print(the_columns_exd)
print(the_columns_ex)

Index(['id', 'name', 'exercise_importance', 'exercise_cost',
       'exercise_recurrance_factor_per_cycle', 'exercise_rpe',
       'exercise_overall_fun_factor', 'exercise_exhaustion_factor',
       'missed_in_cycle_count'],
      dtype='object')
Index(['id', 'name', 'exercise_type', 'muscles_targeted', 'calories_10_min',
       'fun_factor', 'is_compound', 'how_critical'],
      dtype='object')


In [137]:
# Validation the merged columns will be 15
k = set(df_ex.columns)
c = set(df_derived_ex.columns)
for i in c:
    k.add(i)
print(len(k))
k

15


{'calories_10_min',
 'exercise_cost',
 'exercise_exhaustion_factor',
 'exercise_importance',
 'exercise_overall_fun_factor',
 'exercise_recurrance_factor_per_cycle',
 'exercise_rpe',
 'exercise_type',
 'fun_factor',
 'how_critical',
 'id',
 'is_compound',
 'missed_in_cycle_count',
 'muscles_targeted',
 'name'}

In [118]:
# If bicep is in user focus column
"bicep" in df_user[df_user['name'] == 'Mike']['focused_muscle_group'].item().split()

True

In [170]:
def update_weights(row):
    if "bicep" in df_user[df_user['name'] == 'Mike']['focused_muscle_group'].item().split():
        # Update all rows who have bicep in muscle_targeted
        if 'bicep' in row['muscles_targeted'].split():
            id = row['id']
            old_val = df_derived_ex[df_derived_ex['id'] == id]['exercise_importance'].item()
            df_derived_ex.loc[df_derived_ex['id'] == id, 'exercise_importance'] = old_val + 0.37

In [55]:
# Rule trial for mapper
# Poc for mapping iterator
def init():
    df_ex = pd.read_csv("./resources/data/exercise.csv", header=0)
    df_workout = pd.read_csv("./resources/data/workout.csv", header=0)
    df_user = pd.read_csv("./resources/data/user.csv", header=0)
    df_derived_ex = pd.read_csv("./resources/data/exercise_derived.csv", header=0)

    df_derived_ex["exercise_importance"] = 1
    # print(df_derived_ex.head(10))
    return df_ex, df_workout, df_user, df_derived_ex

In [56]:
df_ex, df_workout, df_user, df_derived_ex = init()

In [57]:
from ServiceLayerRecommender import RecommenderLayer
# Rule 1
mapping = {"bicep":0.25, "chest": 0.55, "core": 0.20, "leg": 0.45, "back": 0.55, "shoulder": 0.45, "tricep": 0.37}
for key, value in mapping.items():
    df_ex.apply(lambda row: RecommenderLayer.update_weights_muscle_group_matching(df_derived_ex, row, key, value), axis = 1)

In [58]:
df_derived_ex.head(10)

,id,name,exercise_importance,exercise_cost,exercise_recurrance_factor_per_cycle,exercise_rpe,exercise_overall_fun_factor,exercise_exhaustion_factor,missed_in_cycle_count
0,1,bicep_curl,1.25,NaN,NaN,NaN,NaN,NaN,NaN
1,2,burpee,2.20,NaN,NaN,NaN,NaN,NaN,NaN
2,3,deadlift,2.00,NaN,NaN,NaN,NaN,NaN,NaN
3,4,squat,1.45,NaN,NaN,NaN,NaN,NaN,NaN
4,5,bench_press,2.37,NaN,NaN,NaN,NaN,NaN,NaN
5,6,push_up,2.57,NaN,NaN,NaN,NaN,NaN,NaN
6,7,pull_up,1.80,NaN,NaN,NaN,NaN,NaN,NaN
7,8,lat_pulldown,1.80,NaN,NaN,NaN,NaN,NaN,NaN
8,9,leg_press,1.45,NaN,NaN,NaN,NaN,NaN,NaN
9,10,leg_extension,1.45,NaN,NaN,NaN,NaN,NaN,NaN


In [59]:
df_ex, df_workout, df_user, df_derived_ex = init()

In [60]:
# Rule 2:
'''
If Age in bracket 18 - 30 -> Improve asthetic muscles -> bicep and tricep and shoulder
'''
import Constants
age_lower = 18
age_higher = 40
age_user = int(df_user[df_user['name'] == Constants.USER_NAME]['age'].item())
if age_lower <= age_user and age_user <= age_higher:
    weight_to_improve = 0.55
    mapping = {"bicep":0.25, "core": 0.25, "shoulder": 0.25, "tricep": 0.25}
    for key, value in mapping.items():
        df_ex.apply(lambda row: RecommenderLayer.update_weights_muscle_group_matching(df_derived_ex, row, key, value), axis = 1)

In [61]:
df_derived_ex.head(10)

,id,name,exercise_importance,exercise_cost,exercise_recurrance_factor_per_cycle,exercise_rpe,exercise_overall_fun_factor,exercise_exhaustion_factor,missed_in_cycle_count
0,1,bicep_curl,1.25,NaN,NaN,NaN,NaN,NaN,NaN
1,2,burpee,1.25,NaN,NaN,NaN,NaN,NaN,NaN
2,3,deadlift,1.00,NaN,NaN,NaN,NaN,NaN,NaN
3,4,squat,1.00,NaN,NaN,NaN,NaN,NaN,NaN
4,5,bench_press,1.50,NaN,NaN,NaN,NaN,NaN,NaN
5,6,push_up,1.75,NaN,NaN,NaN,NaN,NaN,NaN
6,7,pull_up,1.25,NaN,NaN,NaN,NaN,NaN,NaN
7,8,lat_pulldown,1.25,NaN,NaN,NaN,NaN,NaN,NaN
8,9,leg_press,1.00,NaN,NaN,NaN,NaN,NaN,NaN
9,10,leg_extension,1.00,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
df_ex, df_workout, df_user, df_derived_ex = init()

In [49]:
# Rule 3:
muscle_group_which_user_is_focusing = df_user[df_user['name'] == Constants.USER_NAME]['focused_muscle_group'].item().split()
for i in muscle_group_which_user_is_focusing:
    weight_to_improve = 0.55
    df_ex.apply(lambda row: RecommenderLayer.update_weights_muscle_group_matching(df_derived_ex, row, i, weight_to_improve), axis = 1)

In [50]:
df_derived_ex.head(10)

,id,name,exercise_importance,exercise_cost,exercise_recurrance_factor_per_cycle,exercise_rpe,exercise_overall_fun_factor,exercise_exhaustion_factor,missed_in_cycle_count
0,1,bicep_curl,1.80,NaN,NaN,NaN,NaN,NaN,NaN
1,2,burpee,1.80,NaN,NaN,NaN,NaN,NaN,NaN
2,3,deadlift,2.10,NaN,NaN,NaN,NaN,NaN,NaN
3,4,squat,1.55,NaN,NaN,NaN,NaN,NaN,NaN
4,5,bench_press,2.05,NaN,NaN,NaN,NaN,NaN,NaN
5,6,push_up,2.30,NaN,NaN,NaN,NaN,NaN,NaN
6,7,pull_up,2.35,NaN,NaN,NaN,NaN,NaN,NaN
7,8,lat_pulldown,2.35,NaN,NaN,NaN,NaN,NaN,NaN
8,9,leg_press,1.55,NaN,NaN,NaN,NaN,NaN,NaN
9,10,leg_extension,1.55,NaN,NaN,NaN,NaN,NaN,NaN


In [62]:
df_ex, df_workout, df_user, df_derived_ex = init()

In [64]:
if df_user[df_user['name'] == Constants.USER_NAME]['goal'].item() == "strength":
    

In [53]:
columns_to_replace_nan_to_zero = ["exercise_importance", "exercise_cost", "exercise_recurrance_factor_per_cycle", "exercise_rpe", "exercise_overall_fun_factor", "exercise_exhaustion_factor", "missed_in_cycle_count"]
for i in columns_to_replace_nan_to_zero:
    df_derived_ex[i].fillna(0, inplace=True)

In [54]:
df_derived_ex.head(10)

,id,name,exercise_importance,exercise_cost,exercise_recurrance_factor_per_cycle,exercise_rpe,exercise_overall_fun_factor,exercise_exhaustion_factor,missed_in_cycle_count
0,1,bicep_curl,1,0.0,0.0,0.0,0.0,0.0,0.0
1,2,burpee,1,0.0,0.0,0.0,0.0,0.0,0.0
2,3,deadlift,1,0.0,0.0,0.0,0.0,0.0,0.0
3,4,squat,1,0.0,0.0,0.0,0.0,0.0,0.0
4,5,bench_press,1,0.0,0.0,0.0,0.0,0.0,0.0
5,6,push_up,1,0.0,0.0,0.0,0.0,0.0,0.0
6,7,pull_up,1,0.0,0.0,0.0,0.0,0.0,0.0
7,8,lat_pulldown,1,0.0,0.0,0.0,0.0,0.0,0.0
8,9,leg_press,1,0.0,0.0,0.0,0.0,0.0,0.0
9,10,leg_extension,1,0.0,0.0,0.0,0.0,0.0,0.0


In [ ]:
# Dump
df_ex.apply(lambda row: update_weights(row), axis=1)
# Join the datasets
merged_df = pd.merge(df_ex, df_derived_ex, on=['id', 'name'])
